In [16]:
from robmob import robot as rbt
from robmob.sensors import GyroSensor, OdometerTicksSensor
import cv2
import numpy as np

import matplotlib.pyplot as plt

import time
from datetime import datetime

import threading

In [3]:
robot_ip = '10.248.251.93'
robot = rbt.Robot(robot_ip)
robot.connect()

In [22]:
DELTA_T = 0.1
NUMBER_OF_STEPS = 50

gyro = GyroSensor()
robot.add_sensor(gyro)
odometer = OdometerTicksSensor()
robot.add_sensor(odometer)

time.sleep(1)
print(gyro.peek_data())
print(odometer.peek_data())

{'t': 1512757504, 'x': 0.8575000000000002, 'y': 0.9625000000000001, 'z': -1.5925}
(1512757504.5111063, -32528, 730)


In [23]:
print(gyro.peek_data())
print(odometer.peek_data())

{'t': 1512757505, 'x': 0.7612500000000002, 'y': 1.0150000000000001, 'z': -1.5312500000000002}
(1512757505.927376, -32528, 730)


In [ ]:
SCALE = 0.5
HEIGHT = 0.14
METERS_PER_PIXEL = (0.292/826.36)/SCALE


def resize(img):
    return cv2.resize(img, None, fx=SCALE, fy=SCALE)


cap = cv2.VideoCapture(1)
# params for ShiTomasi corner detection
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)
# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_frame = resize(old_frame)
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

In [26]:
timestamp = []
gyro_x_data = []
gyro_y_data = []
gyro_z_data = []
odometer_left_data = []
odometer_right_data = []


def data_acquisition(duration):
    
    print(duration)
    global timestamp
    global gyro_x_data
    global gyro_y_data
    global gyro_z_data
    global odometer_left_data
    global odometer_right_data
    
    start_time = time.time()
    stop_time = start_time + duration
    timestamp_time = time.time()
    while timestamp_time < stop_time:
        timestamp_time = time.time()
        gyro_raw_data = gyro.peek_data()
        odometer_raw_data = odometer.peek_data()
        gyro_x_data.append(gyro_raw_data['x'])
        gyro_y_data.append(gyro_raw_data['y'])
        gyro_z_data.append(gyro_raw_data['z'])
        odometer_left_data.append(odometer_raw_data[1])
        odometer_right_data.append(odometer_raw_data[2])
        timestamp.append(odometer_raw_data[0])
        time.sleep(DELTA_T)

In [30]:
diff_x = 0
diff_y = 0
diffs_x = []
diffs_y = []


def optical_flow(duration):
    print("Optical flow thread started")
    global diff_x
    global diff_y
    start_time = time.time()
    stop_time = start_time + duration
    timestamp_time = time.time()
    
    while timestamp_time < stop_time:
        timestamp_time = time.time()
        ret, frame = cap.read()
        if not ret:
            break
        frame = resize(frame)
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        # draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            diff_x += (a - c) / len(good_new)
            diff_y += (b - d) / len(good_new)
            mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
            frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
        img = cv2.add(frame, mask)
        cv2.imshow('frame', img)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)

    print("Optical flow thread finished")

In [28]:
speed = 0.1 #m/s

timestamp = []
gyro_x_data = []
gyro_y_data = []
gyro_z_data = []
odometer_left_data = []
odometer_right_data = []

duration = 10
data_thread = threading.Thread(target=data_acquisition, kwargs={'duration': duration})

data_thread.start()
robot.general_movement(0.1, 0, duration)
data_thread.join()

np.savetxt('data/log-{}.txt'.format(str(datetime.now())),
           np.vstack((timestamp, gyro_x_data, gyro_y_data, gyro_z_data, odometer_left_data, odometer_right_data)))
print("Finish")

10


Finish


In [13]:
robot.general_movement(0.4, np.deg2rad(0), 6)